In [30]:
import csv
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import pandas as pd

df = pd.read_csv('labels.csv')
df = df.query('photo_quality == 1')

df_train = df.loc[(df.is_validation == 0) & (df.is_final_validation == 0)]
df_valid = df.loc[(df.is_validation == 1)]
df_test = df.loc[(df.is_final_validation == 1)]

print(df_train.shape)
print(type(df_train))
array = df_train.to_numpy()
print(array.shape)
print(type(array))

print('oh no')


#img = mpimg.imread(array[0])


            

(4908, 10)
<class 'pandas.core.frame.DataFrame'>
(4908, 10)
<class 'numpy.ndarray'>
oh no


In [31]:
labels_cols = ['is_bee', 'is_wasp', 'is_otherinsect', 'is_other']

y_train = df_train.loc[:,['id']+labels_cols]
y_train.set_index('id', inplace = True)
y_valid = df_valid.loc[:,['id']+labels_cols]
y_valid.set_index('id', inplace = True)
y_test = df_test.loc[:,['id']+labels_cols]
y_test.set_index('id', inplace = True)

print(y_train)

       is_bee  is_wasp  is_otherinsect  is_other
id                                              
1           1        0               0         0
5           1        0               0         0
6           1        0               0         0
7           1        0               0         0
8           1        0               0         0
...       ...      ...             ...       ...
10558       0        0               1         0
10560       0        0               1         0
10562       0        0               1         0
10564       0        0               1         0
10565       0        0               1         0

[4908 rows x 4 columns]


In [32]:
import tensorflow as tf
import numpy as np

IMG_Size = 32
BS = 32

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,)

train_generator = train_gen.flow_from_dataframe(dataframe=df_train, 
                                                  x_col="path", y_col=labels_cols, 
                                                  class_mode="raw", 
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)
ti, tl = train_generator.next()

valid_generator = train_gen.flow_from_dataframe(dataframe=df_valid,
                                                  x_col="path", y_col=labels_cols, 
                                                  class_mode="raw", 
                                                  # class_mode="categorical", 
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)

#Data til test skal heller ikke normaliseres, men vi skalerer det ligeledes  
test_generator = train_gen.flow_from_dataframe(dataframe=df_test,
                                                  x_col="path", y_col=labels_cols, 
                                                  class_mode="raw", 
                                                  shuffle = False,
                                                  target_size=(IMG_Size,IMG_Size), batch_size = BS)

Found 4908 validated image filenames.
Found 1043 validated image filenames.
Found 1084 validated image filenames.


In [42]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pandas as pd

traini, trainl = train_generator.next()
validi, validl = valid_generator.next()
testi, testl = test_generator.next()

print("Shape of x_train pre-processing: ", traini.shape)
print("Shape of y_train pre-processing: ", trainl.shape)

print("Shape of x_val pre-processing: ", validi.shape)
print("Shape of y_val pre-processing: ", validl.shape)

print("Shape of x_test pre-processing: ", testi.shape)
print("Shape of y_test pre-processing: ", testl.shape)

Shape of x_train pre-processing:  (32, 32, 32, 3)
Shape of y_train pre-processing:  (32, 4)
Shape of x_val pre-processing:  (32, 32, 32, 3)
Shape of y_val pre-processing:  (32, 4)
Shape of x_test pre-processing:  (32, 32, 32, 3)
Shape of y_test pre-processing:  (32, 4)


In [43]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout

# Create model architecture
categories = len(labels_cols)


model = Sequential()
model.add(Flatten(input_shape=traini.shape[1:]))
model.add(Dense(1000, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(categories, activation="softmax"))

# Compile the model
model.compile(loss="categorical_crossentropy", optimizer="rmsprop", 
                metrics=["accuracy"])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 3072)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 1000)              3073000   
_________________________________________________________________
dropout_12 (Dropout)         (None, 1000)              0         
_________________________________________________________________
dense_19 (Dense)             (None, 512)               512512    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_20 (Dense)             (None, 4)                 2052      
Total params: 3,587,564
Trainable params: 3,587,564
Non-trainable params: 0
____________________________________________

In [44]:
from keras.callbacks import ModelCheckpoint

mlp_weights_path = "saved_weights/beeswasps_mlp_best.hdf5"

checkpointer_mlp = ModelCheckpoint(filepath=mlp_weights_path, verbose=1, save_best_only=True)
hist_mlp = model.fit(traini, trainl, batch_size=32, 
                   epochs=100, validation_data=(validi, validl),
                   callbacks=[checkpointer_mlp], shuffle=True)

Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 1.5598 - accuracy: 0.2812
Epoch 00001: val_loss improved from inf to 20.12515, saving model to saved_weights\beeswasps_mlp_best.hdf5
1/1 [==============================] - 0s 241ms/step - loss: 1.5598 - accuracy: 0.2812 - val_loss: 20.1252 - val_accuracy: 0.2188
Epoch 2/100
1/1 [==============================] - ETA: 0s - loss: 16.0712 - accuracy: 0.3438
Epoch 00002: val_loss did not improve from 20.12515
1/1 [==============================] - 0s 24ms/step - loss: 16.0712 - accuracy: 0.3438 - val_loss: 32.2240 - val_accuracy: 0.3750
Epoch 3/100
1/1 [==============================] - ETA: 0s - loss: 37.2936 - accuracy: 0.4062
Epoch 00003: val_loss improved from 20.12515 to 10.90046, saving model to saved_weights\beeswasps_mlp_best.hdf5
1/1 [==============================] - 0s 26ms/step - loss: 37.2936 - accuracy: 0.4062 - val_loss: 10.9005 - val_accuracy: 0.4062
Epoch 4/100
1/1 [==============================] - ETA: 0s

In [ ]:
sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(ti)
X_testscaled=sc_X.transform(vi)

reg = MLPRegressor(hidden_layer_sizes=(1,1,1),activation="relu" ,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

y_pred=reg.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pandas as pd

cal_housing = fetch_california_housing()
X = pd.DataFrame(cal_housing.data,columns=cal_housing.feature_names)
y = cal_housing.target
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=1, test_size=0.2)

sc_X = StandardScaler()
X_trainscaled=sc_X.fit_transform(X_train)
X_testscaled=sc_X.transform(X_test)

reg = MLPRegressor(hidden_layer_sizes=(64,64,64),activation="relu" ,random_state=1, max_iter=2000).fit(X_trainscaled, y_train)

y_pred=reg.predict(X_testscaled)
print("The Score with ", (r2_score(y_pred, y_test)))